In [ ]:
!pip install tensorflow-text

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d aminesedkaoui/epinions-ratings-500k
! kaggle datasets download -d aminesedkaoui/epinions-user-item-matrix-75k

In [286]:
from sklearn import preprocessing

In [3]:
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import pandas as pd
import random
from keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [4]:
!unzip "/content/epinions-ratings-500k.zip" -d "/content/"
!unzip "/content/epinions-user-item-matrix-75k.zip" -d "/content/"

Archive:  /content/epinions-ratings-500k.zip
  inflating: /content/epinions_rating_500k.csv  
Archive:  /content/epinions-user-item-matrix-75k.zip
  inflating: /content/epinions_user_item_filled_75K_matrix_100k_leaning.csv  


In [ ]:
df = pd.read_csv('/content/epinions_rating_500k.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df.head()

In [6]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe[itemID].value_counts(sort=False):
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from float to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return user_item,df_user_item

In [7]:
def preprocess_df(df):
  users = df['User_ID'].unique()
  items = df['Item_ID'].unique()
  df_train = df.copy()

  New_User_IDs = random.sample(range(10000,50000),df['User_ID'].nunique())
  New_Item_IDs = random.sample(range(100000,200000),df['Item_ID'].nunique())
  i = 0
  for d in users :
    df_train['User_ID'].replace({d : New_User_IDs[i]}, inplace=True)
    i+=1

  i = 0
  for d in items :
    df_train['Item_ID'].replace({d : New_Item_IDs[i]}, inplace=True)
    i+=1
  
  df_train['one']=df_train['rating'].apply(lambda x: 1 if x==1 else 0)
  df_train['two']=df_train['rating'].apply(lambda x: 1 if x==2 else 0)
  df_train['three']=df_train['rating'].apply(lambda x: 1 if x==3 else 0)
  df_train['four']=df_train['rating'].apply(lambda x: 1 if x==4 else 0)
  df_train['five']=df_train['rating'].apply(lambda x: 1 if x==5 else 0)
  df_train['six']=df_train['rating'].apply(lambda x: 1 if x==6 else 0)

  return df_train

In [8]:
from numpy.core.fromnumeric import size
#GMF

def Create_GMF(num_users,num_items,SIZE_):

  input_userID = layers.Input(shape=[1], name='user_ID')
  input_itemID = layers.Input(shape=[1], name='item_ID')

  user_emb_GMF = layers.Embedding(num_users, SIZE_, name='user_emb_GMF')(input_userID)
  item_emb_GMF = layers.Embedding(num_items, SIZE_, name='item_emb_GMF')(input_itemID)

  u_GMF = layers.Flatten()(user_emb_GMF)
  i_GMF = layers.Flatten()(item_emb_GMF)

  dot_layer = layers.Multiply()([u_GMF, i_GMF])

  out_layer = layers.Dense(6, activation='softmax', name='output')(dot_layer)

  GMF = tf.keras.Model([input_userID, input_itemID], out_layer)
  
  return GMF

def Train_GMF(model,X_train,Y_train,nbrEpochs):
  model.compile(optimizer = 'adam',
                    loss = tf.keras.losses.categorical_crossentropy ,
                    metrics=['accuracy'])
  model.fit(X_train,Y_train,epochs = nbrEpochs)
  return model

def user_item_ID_lists(userIDs,itemIDs):
  i = 0
  item_s = pd.Series()
  user_s = pd.Series()
  for user in userIDs:
    temp = pd.Series(itemIDs)
    item_s = item_s.append(temp)
    temp = []
    temp = [user for item in itemIDs]
    temp = pd.Series(temp)
    user_s = user_s.append(temp)
    
    progress = user_s.size*100/(userIDs.size * itemIDs.size)
   
    if (progress  > 10) & (int(progress) < 25) & (i == 0):
      print("===== 10 % =====")
      i += 1
    elif (progress > 25) & (int(progress) < 35) & (i == 1):
      print("===== 25 % =====")
      i += 1
    elif (progress > 35) & (int(progress) < 50) & (i == 2):
      print("===== 35 % =====")
      i += 1
    elif (progress > 50) & (int(progress) < 65) & (i == 3):
      print("===== 50 % =====")
      i += 1
    elif (progress > 65) & (int(progress) < 75) & (i == 4):
      print("===== 65 % =====")
      i += 1
    elif (progress > 75) & (int(progress) < 80) & (i == 5):
      print("===== 75 % =====")
      i += 1
    elif (progress > 80) & (int(progress) < 90) & (i == 6):
      print("===== 80 % =====")
      i += 1
    elif (progress > 90) & (int(progress) < 95) & (i == 7):
      print("===== 90 % =====")
      i += 1
    elif (progress > 95) & (i == 8):
      print("===== 95 % =====")
      i += 1

  return user_s,item_s

def Fill_Cf_Matrix(model,userList,itemList,userIDs,itemIDs):

  prediction = model.predict([userList,itemList],verbose = 0)
  i = 0
  row = []
  matrix = []
  print("pred done")

  while i < userList.shape[0]:
    result = np.where(prediction[i] == np.amax(prediction[i]))[0][0] + 1
    row.append(result)
    if len(row) == itemIDs.size:
      matrix.append(row)
      row = []
       
    i += 1

  matrix_arr = np.array(matrix)
  dataframe = pd.DataFrame(matrix_arr, index = userIDs, columns = itemIDs)
  # for user in userIDs:
  #   user_s = pd.Series(user)
  #   for item in itemIDs:
  #     item_s = pd.Series(item)
  #     if dataframe.loc[user][item] == 0:
  #       prediction = model.predict([user_s,item_s],verbose = 0)
  #       result = np.where(prediction[0] == np.amax(prediction[0]))[0][0] + 1
  #       dataframe.loc[user][item] = result
  return dataframe


In [9]:
df_train = preprocess_df(df)

In [ ]:
df_train

In [320]:
GMF = Create_GMF(df_train['User_ID'].max() + 1,df_train['Item_ID'].max() + 1,64)

In [319]:
df_train_sample = df_train[:20000]
X_train, X_test,y_train,y_test= train_test_split(df_train_sample[['User_ID','Item_ID']],df_train_sample[['one','two','three','four','five','six']],stratify=df_train_sample[['one','two','three','four','five','six']])
#X_train, X_test,y_train,y_test= train_test_split(df_train[['User_ID','Item_ID']],df_train['rating'],stratify=df_train['rating'])

In [322]:
GMF_trained = Train_GMF(GMF,[X_train['User_ID'],X_train['Item_ID']],y_train,5)

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.1135 - accuracy: 0.9728
Epoch 2/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0687 - accuracy: 0.9858
Epoch 3/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0410 - accuracy: 0.9939
Epoch 4/5
469/469 [==============================] - 5s 12ms/step - loss: 0.0246 - accuracy: 0.9979
Epoch 5/5
469/469 [==============================] - 5s 12ms/step - loss: 0.0149 - accuracy: 0.9994


In [323]:
user_item_matrix , df_mat = DatasetToUserItemDataFrame(df_train_sample,'User_ID','Item_ID','rating')

In [ ]:
GMF_trained.evaluate([X_test['User_ID'],X_test['Item_ID']],y_test)

In [ ]:
New_User_IDs = df_train_sample.User_ID.unique()
New_Item_IDs = df_train_sample.Item_ID.unique()
New_User_IDs

In [ ]:
New_Item_IDs.size * New_User_IDs.size

In [ ]:
user_s,item_s = user_item_ID_lists(New_User_IDs,New_Item_IDs)

In [329]:
df_mat_filled = Fill_Cf_Matrix(GMF_trained,user_s,item_s,New_User_IDs,New_Item_IDs)

pred done


In [ ]:
df_mat_filled

In [12]:
df_mat_filled1 = pd.read_csv('/content/epinions_user_item_filled_75K_matrix_100k_leaning.csv')

In [ ]:
df_mat

In [ ]:
df_mat_filled1

In [267]:

df_mat_sample.replace(0,' ', inplace = True)


In [332]:
df_mat_sample =df_mat_filled

In [ ]:
df_mat_sample.head()

In [135]:
def to_binary(x):
  if (x > 0):
    x = 1
  return x

In [141]:
def df_to_binary(df):
  for col in df.columns:
    df[col] = df[col].apply(to_binary)
  return df

In [ ]:
df_to_binary(df_mat)
df_to_binary(df_mat_sample)

In [ ]:
df_mat_sample

In [334]:
#Autoencoder

encoder_input = layers.Input(shape=(df_mat_sample.shape[1]),name='user_item')
flat = layers.Flatten()(encoder_input)
#dropout = layers.Dropout(.2)(flat)
hid_encoder = layers.Dense(128,activation="relu")(flat)
# hid_encoder1 = layers.Dense(256,activation="relu")(hid_encoder)
# hid_encoder2 = layers.Dense(128,activation="relu")(hid_encoder1)
# hid_encoder3 = layers.Dense(10,activation="relu")(hid_encoder2)
encoder_output = layers.Dense(64,activation="relu")(hid_encoder)

decoder_input = layers.Dense(128,activation="relu")(encoder_output)
# decoder_hidden1 = layers.Dense(256,activation="relu")(decoder_input)
# decoder_hidden2 = layers.Dense(512,activation="relu")(decoder_hidden1)
# decoder_hidden3 = layers.Dense(25,activation="relu")(decoder_hidden2)
decoder_output = layers.Dense(df_mat_sample.shape[1],activation="relu")(decoder_input)

autoencoder = tf.keras.Model(inputs = encoder_input, outputs = decoder_output)

In [100]:
def rmse (y_true,y_pred):
  y_pred = tf.cast(y_pred, tf.float32)
  y_true = tf.cast(y_true, tf.float32)
  rmse = y_true - y_pred
  rmse = tf.square(rmse)
  rmse = tf.math.reduce_mean(rmse)
  rmse = tf.math.sqrt(rmse)
  return rmse

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

In [335]:
autoencoder.compile(optimizer ='adam',
                    loss = tf.keras.losses.MeanAbsoluteError(),
                    metrics=['accuracy'])

In [ ]:
autoencoder.summary()

In [337]:
X_train,X_test,y_train,y_test = train_test_split(df_mat,df_mat_sample)

In [ ]:
X_train

In [ ]:
autoencoder.fit(X_train, y_train, epochs = 2000)

In [ ]:
autoencoder.evaluate(X_test,y_test)

In [ ]:
df_mat

In [355]:
mat_pred = autoencoder.predict(df_mat)

91/91 [==============================] - 0s 2ms/step


In [ ]:
mat_pred

In [357]:
df_mat_pred = pd.DataFrame(mat_pred, index = df_mat_sample.index, columns = df_mat_sample.columns)

In [ ]:
df_mat_pred.max()

In [364]:
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
d = scaler.fit_transform(df_mat_pred)
scaled_df = pd.DataFrame(d,index = df_mat_pred.index,columns = df_mat_pred.columns)
scaled_df

,142900,193193,149668,109569,190973,130885,102356,140724,150894,121811,...,148093,161328,162786,127922,182458,162746,111242,179615,196661,196174
24182,0.739560,0.690588,0.394466,0.393343,0.724470,0.457326,0.722702,0.545093,0.0,0.661743,...,0.718401,0.620500,0.747101,0.376430,0.443698,0.708718,0.604317,0.359654,0.382450,0.664147
10293,0.741335,0.691571,0.408539,0.376687,0.726362,0.458363,0.723970,0.546680,0.0,0.663787,...,0.720085,0.621630,0.749273,0.384166,0.446956,0.709549,0.608099,0.361604,0.376744,0.665603
39186,0.740762,0.691124,0.401990,0.352926,0.725216,0.456493,0.723390,0.545573,0.0,0.662940,...,0.719465,0.620951,0.748868,0.379751,0.445569,0.709728,0.604250,0.360029,0.368365,0.664972
36361,0.740440,0.691350,0.393258,0.354361,0.724953,0.454975,0.723332,0.546809,0.0,0.663099,...,0.719472,0.621969,0.748358,0.391833,0.448424,0.709648,0.605230,0.359348,0.349215,0.665215
48234,0.739536,0.690577,0.401818,0.419009,0.724415,0.455279,0.722702,0.544563,0.0,0.661844,...,0.718436,0.620313,0.747451,0.382252,0.443615,0.708749,0.602944,0.359200,0.363024,0.664120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49603,0.490176,0.465485,0.683034,0.576619,0.483633,0.899976,0.490776,0.920524,0.0,0.442354,...,0.480517,0.924342,0.594342,0.667836,0.443056,0.476412,0.928867,0.918885,0.756639,0.449879
22328,0.740901,0.691945,0.552346,0.520014,0.725198,0.458674,0.723261,0.544917,0.0,0.662895,...,0.720137,0.621196,0.748950,0.410298,0.446678,0.710601,0.605720,0.358838,0.541753,0.665597
49900,0.740901,0.691945,0.552346,0.520014,0.725198,0.458674,0.723261,0.544917,0.0,0.662895,...,0.720137,0.621196,0.748950,0.410298,0.446678,0.710601,0.605720,0.358838,0.541753,0.665597
44339,0.740901,0.691945,0.552346,0.520014,0.725198,0.458674,0.723261,0.544917,0.0,0.662895,...,0.720137,0.621196,0.748950,0.410298,0.446678,0.710601,0.605720,0.358838,0.541753,0.665597


In [ ]:
res = np.where(df_mat_pred.iloc[:,:].values == 1 )
res[0].size